In [8]:
import os
import numpy as np
from skimage import io, img_as_ubyte
from tqdm import tqdm
%run my_functions.ipynb

In [17]:
faces_path = "../data/Faces/"
labels_path = "../data/labels.csv"
names_path = "../data/filenames.txt"
file_data_path = "../files"
data_set_gray_npy = os.path.join(file_data_path, 'data_gray.npy')
data_set_rgb_npy = os.path.join(file_data_path, 'data_rgb.npy')
labels_csv_path = os.path.join(file_data_path, 'data_labels.csv')

In [18]:
# Get labels as pandas object from labels.csv
labels = get_labels_df(
    labels_path=labels_path, names_path=names_path)
labels

,actual_filename,image_no,age,gender,race
0,100_0_0_20170112213500903.jpg.chip.jpg,0,100,0,0
1,100_0_0_20170112215240346.jpg.chip.jpg,1,100,0,0
2,100_1_0_20170110183726390.jpg.chip.jpg,2,100,1,0
3,100_1_0_20170112213001988.jpg.chip.jpg,3,100,1,0
4,100_1_0_20170112213303693.jpg.chip.jpg,4,100,1,0
...,...,...,...,...,...
23700,9_1_3_20161220222856346.jpg.chip.jpg,23700,9,1,3
23701,9_1_3_20170104222949455.jpg.chip.jpg,23701,9,1,3
23702,9_1_4_20170103200637399.jpg.chip.jpg,23702,9,1,4
23703,9_1_4_20170103200814791.jpg.chip.jpg,23703,9,1,4


In [11]:
# Put age in bins
age_bin_width_years = 10
ages = labels.age.values//age_bin_width_years  # bin_width fully lived
labels.drop(['age'], axis=1, inplace=True)
labels['age_bin'] = ages

In [13]:
# Open a random image to get dimensions
h, w = get_dimensions_from_an_image(faces_path, 0, as_gray=True)

# Save grayscale images
dirs = os.listdir(faces_path)
X = np.empty((len(dirs), h*w), dtype=np.ubyte)

for i in tqdm(range(len(dirs))):
    a = io.imread(faces_path+f"{i}.jpg", as_gray=True)
    a = img_as_ubyte(a)
    X[i, :] = a.reshape(1, -1)
del a

100%|█████████████████████████████████████████████| 23705/23705 [00:28<00:00, 824.92it/s]


In [14]:
labels_loaded = labels.copy()

In [15]:
# Save labels and data matrix X to disk
np.save(data_set_gray_npy, X)
labels_loaded = labels.copy()
labels_loaded.to_csv(labels_csv_path, index=False)

del X

In [16]:
# RGB
X_rgb = np.empty((len(dirs), h*w, 3), dtype=np.ubyte)

for i in tqdm(range(len(dirs))):
    a = io.imread(faces_path+f"{i}.jpg")
    X_rgb[i, :, :] = a.reshape(1, -1, 3)
del a
# Save data to disk
np.save(data_set_rgb_npy, X_rgb)
del X_rgb

100%|████████████████████████████████████████████| 23705/23705 [00:13<00:00, 1784.64it/s]
